In [1]:
# Выполнение одного набора xgbooster (JPG, либо TIF) (2017-07-03)

In [2]:
import sys,os,datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.metrics as skm
from __future__ import print_function

In [3]:
print(np.__version__);
print(pd.__version__);

1.12.1
0.19.2


In [4]:
import  cv2 as cv
cv.__version__

'3.2.0'

In [5]:
import xgboost as xgb

/home/boba/anaconda2/envs/tensorflow-gpu/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [6]:
sys.path.append('../Python')
from helper import paths_input, formImExt, formImHist
from estimate import confusion_matrix, getConfusion, getRocAUC, getProb01, getProbX01, getTh, estimateResult

In [7]:
trLabels,trDirTIF,trDirJPG,teDirTIF,teDirJPG = paths_input()
trWork, teWork = '../Work/Train', '../Work/Test'
print (trWork, teWork)
print (trLabels,trDirTIF,trDirJPG,teDirTIF,teDirJPG)

../Work/Train ../Work/Test
../Data/train_v2.csv ../Data/train-tif-v2 ../Data/train-jpg ../Data/test-tif-v2 ../Data/test-jpg-v2


In [8]:
labels_df = pd.read_csv(trLabels)
labels_df.head(2)

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water


In [9]:
# Build list with unique labels
label_list = []
for tag_str in labels_df.tags.values:
    labels = tag_str.split(' ')
    for label in labels:
        if label not in label_list:
            label_list.append(label)

In [10]:
# Add features (0 or 1) for every label as dataframe field 
for label in label_list:
    labels_df[label] = labels_df['tags'].apply(lambda x: 1 if label in x.split(' ') else 0)
#labels_df.head()

In [11]:
weather_labels = ['clear', 'partly_cloudy', 'haze', 'cloudy']
land_labels = ['primary', 'agriculture', 'water', 'cultivation', 'habitation' ]
rare_labels = [l for l in label_list if labels_df[label_list].sum()[l] < 2000]
#rare_labels              

In [12]:
labels = label_list; #weather_labels;
nameList =labels_df[labels_df[labels].sum(axis=1)>0].image_name.tolist(); len(nameList)
labelList=labels_df[labels_df[labels].sum(axis=1)>0][labels].as_matrix();
#labelList[:6,:]
labels_df.head(2)

,image_name,tags,haze,primary,agriculture,clear,water,habitation,road,cultivation,slash_burn,cloudy,partly_cloudy,conventional_mine,bare_ground,artisinal_mine,blooming,selective_logging,blow_down
0,train_0,haze primary,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,train_1,agriculture clear primary water,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Расчет решений, оценка результативности и генерация смеси квадратно-картофелным методом
#  Выбираются решения для лучшего по train и объединяются в окончательное решение
#
#  2017-07-08
#
#  Результативность странная:
#    1. Лучшую результативность по train дают equalizeHist - решения. Но на реальных данных они дают серьезный регресс
#    2. Лучшую результативность на test дает обычная схема без улучшений, следующая gauss схема. На train немного наоборот.
#    3. Например jpg256 norma=0.85949, gauss=0.85670, equalizeHist=0.79159, E+G=0.78989
#    4. Смесь лучших из n,g,e,g+e = 0.81241
#    5. Смесь gauss и norma = 0.85767.Эффект незначительный, небольшое снижение от norma, хотя по train значительный на 2000 
#
#  Вывод: Методика не перспективна. Прекращено. В архиве оставлено на случай возникновения новых идей.
#
#

In [18]:
param = {
         'objective':'reg:logistic', 
         #'num_class':17,
         #'eta':0.05, 
         #'max_depth':9, 'subsample':0.8, 'colsample_bytree':0.8, 
         #'metrics':'rmse'
         'eval_metric':['auc','rmse']
        }
param['silent']=True

In [20]:
# Шаблоны имен файлов для сохранения моделей xgb
fModel1 = '../Data-XGB/Models/model-'
fModel2 = str(binsData)+'-'+jpgtif+'.xgb'

In [21]:
#
# Построение списка решений из набора train, test, models (сейчас расчитано на jpg только)
#

binsData = 64
jpgtif   = '.jpg'

res      = []

for xx in ['','-G'] : #,'-E','-GE'] :

    trX      = np.load('../Data-XGB/Datas/train-'+str(10000+binsData)+jpgtif+'-XX'+xx+'.npy')
    trY      = np.load('../Data-XGB/Datas/train-'+str(10000+binsData)+jpgtif+'-YY'+xx+'.npy')

    fModel2 = str(binsData)+'-'+jpgtif+xx+'.xgb'
    
    # load xgb models into files
    bstNull = xgb.train(param,dtrain,0)
    bstList0= []
    for i in range(trY.shape[1]) :
        bstList0.append(bstNull.copy())
        bstList0[-1].load_model(fModel1+str(100+i)+fModel2)
    bstList = bstList0
    del bstList0
    
    # Построение решения
    trYP  = buildPredictX(trX,bstList)
    trYPP = trYP.copy() # вероятностный массив
    trYP  = getProb01(trYP)
    
    # Уровни прохождения
    trPPP,_ = getTh (trY,trYPP)
    trYP  = getProbX01(trYPP,trPPP)
    
    del trX
    
    trOX     = np.load('../Data-XGB/Datas/test-'+str(10000+binsData)+jpgtif+'-XX'+xx+'.npy')
    trOY     = np.load('../Data-XGB/Datas/test-'+str(10000+binsData)+jpgtif+'-YY'+xx+'.npy')
    
    trOP  = buildPredictX(trOX,bstList)
    trOPP = trOP.copy() # вероятностный массив
    trOP  = getProbX01(trOP,trPPP)
    
    del trOX

    res.append((xx,trYP,trOP,trOY))

In [22]:
#
# Техническая загрузка для автоматического расчета размеров
#
trOX     = np.load('../Data-XGB/Datas/test-'+str(10000+binsData)+jpgtif+'-XX.npy')
len(res)

2

In [23]:
# Расчет результативности моделей и выемка потерь по признакам на основе статистики train и полученного решения для train 
temp_train = [estimateResult(trY,trYP)[:,8] for _,trYP,_,_ in res]

In [24]:
temp_loss_train = np.array(temp_train,dtype=np.uint16); temp_loss_train
#temp_loss_train.sum(axis=0)

array([[ 820,  451, 3565, 1014, 1981, 1499, 3378, 2301,  107,  140,  393,
           0,  498,    0,  191,  207,   44],
       [ 707,  288, 3157,  668, 2565, 1511, 3374, 2845,   41,  129,   77,
           9,  282,    0,  189,   81,   27]], dtype=uint16)

In [25]:
# Расчет таблицы перестановок из полученной статистики потерь для признаков
permute = [temp_loss_train[:,i].argmin() for i in range(17)]; permute

[1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1]

In [26]:
# Общие потери по каждому наборуу по все признакам (простая сумма)
temp_loss_train.sum(axis=1)

array([16589, 15950], dtype=uint64)

In [27]:
#
# Построение перемешенного решения и оценка его результативности
#

trYNew, trONew = np.zeros(trY.shape,dtype=np.uint8), np.zeros((trOX.shape[0],trY.shape[1]),dtype=np.uint8)
##print(res[1].shape)
for ii in range(trY.shape[1]) :
    xx = permute[ii]
    trYNew[:,ii] = res[xx][1][:,ii]
    trONew[:,ii] = res[xx][2][:,ii]
    
temp_new = estimateResult(trY,trYNew,printOK=True)    

0 acc=0.982534153512 roc=0.896990088741 not=707 yes=2153 no=37619 true=39772 all-1-0=(0.7983-0.9957)
1 acc=0.992885199733 roc=0.97504753533 not=288 yes=37361 no=2830 true=40191 all-1-0=(0.9959-0.9541)
2 acc=0.922008942909 roc=0.907442596098 not=3157 yes=10717 no=26605 true=37322 all-1-0=(0.8702-0.9446)
3 acc=0.983497616048 roc=0.975816861988 not=668 yes=28283 no=11528 true=39811 all-1-0=(0.9948-0.9568)
4 acc=0.951061043998 roc=0.897284351458 not=1981 yes=6021 no=32477 true=38498 all-1-0=(0.8124-0.9821)
5 acc=0.962968452778 roc=0.866208087503 not=1499 yes=2738 no=36242 true=38980 all-1-0=(0.7481-0.9843)
6 acc=0.916648138541 roc=0.859739424279 not=3374 yes=6175 no=30930 true=37105 all-1-0=(0.7651-0.9544)
7 acc=0.943155710368 roc=0.816463032599 not=2301 yes=2927 no=35251 true=38178 all-1-0=(0.6538-0.9791)
8 acc=0.998987129129 roc=0.920953302053 not=41 yes=176 no=40262 true=40438 all-1-0=(0.8421-0.9998)
9 acc=0.99681316238 roc=0.987229962426 not=129 yes=2040 no=38310 true=40350 all-1-0=(0.

In [29]:
# Полученная результатвность смешенного решения
temp_new[:,-1].sum()

14801.0

In [30]:
# Построение результата для нового решения
# !!!  Возможно не решен вопрос о разных порядках файлов, это мб актуально для разных типов jpg-tif
# !!!  Данная проблема решена в модуле JPG_TIF в конце

In [32]:
#trZ = res[3][2] # -- equalizeHist lost=19090 #trONew
trZ = trONew

In [33]:
temp_list = []
for i in range(trZ.shape[0]) :
    temp = [labels[ii] for ii in range(trZ.shape[1]) if trZ[i,ii]==1];
    temp = ' '.join(temp)
    temp_list.append([trOY[i],temp])
temp_list.sort(cmp=lambda x,y: cmp(int(x[0].partition('_')[2]),int(y[0].partition('_')[2])) if (x[0].partition('_')[0]==y[0].partition('_')[0]) else cmp(y[0].partition('_')[0],x[0].partition('_')[0]))


In [35]:
temp_list[0:5]

[['test_0', 'primary clear'],
 ['test_1', 'primary clear'],
 ['test_2', 'primary partly_cloudy'],
 ['test_3', 'primary clear cultivation'],
 ['test_4', 'cloudy']]

In [36]:
rrr=pd.DataFrame(temp_list,columns=['image_name','tags']); rrr.head(); 
suffixDT = (datetime.datetime.now()).strftime('%Y-%m-%d-%H-%M-%S'); print(suffixDT)
rrr.to_csv('../Result/vss'+suffixDT+'.csv',index=False);

2017-07-08-21-32-26


In [276]:
rrr.head()

,image_name,tags
0,test_0,primary clear
1,test_1,primary clear
2,test_2,primary partly_cloudy
3,test_3,primary clear
4,test_4,primary partly_cloudy


In [ ]:
# The End